<a href="https://colab.research.google.com/github/nathvegi/imersao_alura25/blob/main/Projeto_Multi_Agente_Imers%C3%A3o_Alura_IA_%2B_Gemini_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip -q install google-genai

In [ ]:
# Configuração de API KEY do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configuração do Cliente da SDK do Gemini

from google import genai

client = genai.Client()

modelo = "gemini-2.0-flash"

In [ ]:
# Instalação do Framework ADK de agentes Google

!pip install -q google-adk


In [ ]:
from typing import Union, List
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import base64

warnings.filterwarnings("ignore")

In [ ]:
def call_agent(agent: Agent, image_bytes: Union[bytes, None] = None, message_text: Union[str, None] = None) -> str:
    """ Chama um agente com bytes de imagem ou texto.

    Args:
        agent: O agente a ser chamado.
        image_bytes: Os bytes da imagem.
        message_text: O texto da mensagem (opcional).

    Returns:
        O texto da resposta final do agente.
    """

    session_service = InMemorySessionService()

    content_parts: List[types.Part] = []
    if message_text:
        content_parts.append(types.Part(text=message_text))
    if image_bytes:
        # Codifica os bytes da imagem em base64
        encoded_image = base64.b64encode(image_bytes).decode('utf-8')
        # Cria a Parte com os dados da imagem embutidos (inline_data)
        content_parts.append(types.Part(
            inline_data=types.Blob(mime_type="image/jpeg", data=encoded_image)
        ))

    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=content_parts)

    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
print("✨ 🚀 Descrições de Produtos Turbinadas com SEO e IA: Sistema de 3 Agentes - Iniciando agora! ✨\n")

imagem_produto_bytes = None
caminho_para_imagem = ""

while not imagem_produto_bytes:
    caminho_para_imagem = input("▶️ Por favor, digite o CAMINHO do arquivo da imagem que você carregou (obrigatório): ")
    if not caminho_para_imagem.strip():
        print("⚠️ O caminho da imagem é obrigatório. Por favor, insira o caminho.")
        continue  # Volta para o início do loop para solicitar o caminho novamente

    try:
        # Abrir o arquivo de imagem no modo binário de leitura ('rb')
        with open(caminho_para_imagem, 'rb') as f:
            imagem_produto_bytes = f.read()
        print(f"✅ Imagem '{caminho_para_imagem}' carregada com sucesso!")
    except FileNotFoundError:
        print(f"❌ Erro: O arquivo '{caminho_para_imagem}' não foi encontrado. Por favor, verifique o caminho e tente novamente.")
    except Exception as e:
        print(f"❌ Ocorreu um erro ao ler o arquivo: {e}. Por favor, tente novamente.")

# --- Obter informações textuais adicionais do usuário ---
info_textual_adicional = input("❓ Por favor, digite informações adicionais sobre o produto (ou deixe em branco se não houver): ")

✨ 🚀 Descrições de Produtos Turbinadas com SEO e IA: Sistema de 3 Agentes - Iniciando agora! ✨

▶️ Por favor, digite o CAMINHO do arquivo da imagem que você carregou (obrigatório): /content/pijama.jpg
✅ Imagem '/content/pijama.jpg' carregada com sucesso!
❓ Por favor, digite informações textuais adicionais sobre o produto (ou deixe em branco se não houver): Malha 100% Algodão, Produto oficial Licenciado da Warner


In [ ]:
##########################################
# --- Agente 1: Analista de Imagem --- #
##########################################
def agente_imagem(imagem_produto_bytes):
    analista_imagem = Agent(
        name="agente_imagem",
        model="gemini-2.0-flash",
        description="Agente que processa a imagem com a SDK do Gemini e extrai as características visuais.",
        instruction="""
        Você é um agente de imagem. A sua tarefa é analisar a imagem deste produto e descrever suas características visuais principais,
        incluindo cor, formato, material aparente e quaisquer detalhes de design ou textura.
        Liste as características de forma concisa. Você deve ignorar o que o modelo estiver calçando, porque o produto a ser vendido é apenas o pijama.
        Você precisa considerar se o modelo é adulto ou infantil, feminino ou masculino para informar que o produto é adulto ou infantil, feminino ou masculino.
        """
    )

    # Passa a imagem diretamente para o argumento image_bytes
    caracteristicas_visuais = call_agent(analista_imagem, image_bytes=imagem_produto_bytes, message_text="Por favor, descreva esta imagem:")

    return caracteristicas_visuais

In [ ]:
#######################################################
# --- Agente 2: Analista de Texto e Enriquecimento --- #
#######################################################
def agente_analista_texto(caracteristicas_visuais, info_textual_adicional):
    analista_texto = Agent(
        name="agente_analista_texto",
        model="gemini-2.0-flash",
        description="Agente que combina características visuais com informações textuais para enriquecer a descrição do produto.",
        instruction="""
        Você é um analista de imagem enriquecido. Sua tarefa é combinar as características visuais do produto,
        fornecidas anteriormente, com as informações textuais adicionais, fornecidas pelo usuário.
        Use ambas as fontes de informação para criar uma descrição completa e detalhada do produto.
        Inclua detalhes que não eram visíveis apenas na imagem, mas que foram mencionados nas informações textuais.
        Formule uma descrição fluida e informativa que combine todos os dados disponíveis.

        """

    )

    entrada_do_agente_analista_texto = f"""
    Características Visuais: {caracteristicas_visuais}

    Informações Textuais Adicionais: {info_textual_adicional}

    Por favor, crie uma descrição enriquecida do produto com base nessas informações.
    """

    # Executa o agente
    descricao_enriquecida = call_agent(analista_texto, message_text=entrada_do_agente_analista_texto)
    return descricao_enriquecida

In [ ]:
################################################################
# --- Agente 3: Otimizador de SEO e Redator de Descrições --- #
################################################################
def agente_otimizador_redator(descricao_preliminar):
    otimizador_redator = Agent(
        name="agente_otimizador_redator",
        model="gemini-2.0-flash",
        description="Agente que otimiza descrições de produtos para SEO e vendas online.",
        instruction="""
        Você é um Analista Otimizador de SEO e Redator de Descrições especializado em e-commerce.
        Sua tarefa é pegar a descrição preliminar de um produto e otimizá-la para vendas online e motores de busca (SEO).

        Para otimizar a descrição:
        1. Analise a descrição preliminar para entender o produto e suas características.
        2. Pense em palavras-chave relevantes que os clientes usariam para buscar este tipo de produto online.
        3. Incorpore naturalmente essas palavras-chave ao longo da descrição, especialmente no início.
        4. Use linguagem persuasiva focada nos benefícios para o cliente. Destaque como o produto resolve um problema ou atende a uma necessidade.
        5. Mantenha a descrição concisa e fácil de ler. Não use bullet points e gere frases curtas se apropriado.

        Crie a descrição otimizada para SEO e vendas.
            """

    )
    entrada_do_agente_otimizador_redator = f"""
    Descrição Preliminar: {descricao_preliminar}

    Por favor, otimize esta descrição para SEO e vendas online.
    """
    # Executa o agente
    descricao_otimizada = call_agent(otimizador_redator, message_text=entrada_do_agente_otimizador_redator)
    return descricao_otimizada

In [ ]:
# Lógica do sistema de agentes ##################################
print(f"Perfeito! Aguarde enquanto processamos a imagem: {caminho_para_imagem}")

imagens_analisadas = agente_imagem(imagem_produto_bytes)
print("\n--- Resultado do Agente 1 (Analista de Imagem) ---\n")
display(to_markdown(imagens_analisadas))
print("---------------------------------")

descricao_imagem = agente_analista_texto(imagens_analisadas, info_textual_adicional)
print("\n--- Resultado do Agente 2 (Analista de Texto e Enriquecimento) ---\n")
display(to_markdown(descricao_imagem))
print("---------------------------------")

descricao_final = agente_otimizador_redator(descricao_imagem)
print("\n--- Resultado do Agente 3 (Otimizador de SEO e Redator de Descrições) ---\n")
display(to_markdown(descricao_final))
print("---------------------------------")

Perfeito! Aguarde enquanto processamos a imagem: /content/pijama.jpg

--- Resultado do Agente 1 (Analista de Imagem) ---



> Aqui estão as características visuais principais do pijama infantil masculino na imagem:
> 
> *   A camiseta é de cor marrom e apresenta uma estampa grande do personagem Scooby-Doo na frente.
> *   O shorts possui um padrão xadrez em tons de branco, amarelo e azul.
> *   O conjunto é feito de um material macio e confortável.


---------------------------------

--- Resultado do Agente 2 (Analista de Texto e Enriquecimento) ---



> Aqui está uma descrição enriquecida do produto, combinando as características visuais e as informações textuais:
> 
> Este adorável pijama infantil masculino apresenta uma camiseta marrom com uma estampa vibrante do personagem Scooby-Doo, perfeita para os fãs do famoso desenho animado. O shorts, com um padrão xadrez moderno em tons de branco, amarelo e azul, complementa o visual de forma divertida e charmosa. Confeccionado em malha 100% algodão, este pijama oferece um toque suave e aconchegante, garantindo o máximo de conforto para uma noite de sono tranquila. Além disso, este é um produto oficial licenciado da Warner Bros., assegurando a qualidade e autenticidade do design. Ideal para noites agradáveis e momentos de diversão em casa.
> 


---------------------------------

--- Resultado do Agente 3 (Otimizador de SEO e Redator de Descrições) ---



> **Pijama Infantil Scooby-Doo Warner Bros. - Conforto e Diversão para Noites Tranquilas!**
> 
> Seu pequeno fã de Scooby-Doo vai adorar este pijama infantil masculino! Composto por uma camiseta marrom com estampa vibrante do famoso personagem e um shorts xadrez moderno em tons de branco, amarelo e azul, este pijama é perfeito para noites aconchegantes e cheias de diversão.
> 
> Confeccionado em malha 100% algodão, o pijama proporciona um toque suave e garante o máximo de conforto para uma noite de sono tranquila. Além de lindo e divertido, este é um produto oficial licenciado da Warner Bros., garantindo a qualidade e autenticidade que seu filho merece. Ideal para noites agradáveis e momentos de alegria em casa.


---------------------------------
